This notebook is meant for the CLUSTER version of analysis, for the purposes of creating an entropy profile that mimics the Phoenix cluster and similar clusters that lack an AGN that heats against the cooling flow.

In [ ]:
import random
import statistics
import matplotlib.pyplot as plt
import numpy as np
#from scipy.integrate import odeint
#from scipy.stats import linregress
from scipy.interpolate import interp1d
from scipy.integrate import solve_ivp
from scipy.constants import k as k_B  # Boltzmann constant in J/K
from scipy.constants import G, proton_mass
from functools import partial
import lab_functions_1 as lf
from numba import njit
mp_g = proton_mass*1e3
k_Bcgs = k_B*1e7
mu = 0.6
gamma = 5/3
ktc = 3.0857e21
etkv = 6.2415*10**8
kevtk = 1.16*10**7
z = 0.597
Mdot1 = 6.30391e25

In [ ]:
radii_grid = np.linspace(1*ktc, 16000*ktc, 80000)
vc_grid = np.array([lf.vcgrab(r, z, 3e12, 2e15) for r in radii_grid])
vc_interp = interp1d(radii_grid, vc_grid, kind='cubic', fill_value='extrapolate')

In [ ]:
from scipy.integrate import cumulative_trapezoid
r_grid = np.geomspace(ktc*0.01, 10000000 * ktc, 100000)  # finer grid = better accuracy
vc2_over_r = np.array([(lf.vcgrab(r, z, 3e12, 2e15 )**2 / r) for r in r_grid])

phi_cumint = cumulative_trapezoid(vc2_over_r, r_grid, initial=0.0)

phi_values = -(phi_cumint[-1] - phi_cumint)

phi_interp = interp1d(r_grid, phi_values, kind='cubic', fill_value="extrapolate")

def phi(r):
    """Interpolated gravitational potential at any r."""
    return phi_interp(r)

In [ ]:
print(phi(500*ktc))

In [ ]:
def dlnTdlnrcalc(R_sonic, x, T_sonic_point, Lambdatype, pr=True): 
    if Lambdatype == "W":
        dlnL1 = np.log(lf.Lambdacalc(np.log10(T_sonic_point), R_sonic, "a", 1))
        dlnL2 = np.log(lf.Lambdacalc(np.log10(T_sonic_point*1.01), R_sonic, "a", 1))
    else:
        dlnL1 = np.log(lf.Lambdacalc(np.log10(T_sonic_point), R_sonic, Lambdatype, 1))
        dlnL2 = np.log(lf.Lambdacalc(np.log10(T_sonic_point*1.01), R_sonic, Lambdatype, 1))
    
    dlnT1 = np.log(T_sonic_point)
    dlnT2 = np.log(T_sonic_point*1.01)
    dlnLambda_dlnT = (dlnL2 - dlnL1)/(dlnT2 - dlnT1)


    dlnvc_dlnR = lf.dVcdrfunc(R_sonic, z, 3e12, 2e15)
    
    #solve quadratic equation    
    b = 29/6.*x - 17/6. + 1/3.*(1.-x)*(dlnLambda_dlnT)
    c = 2/3.*x*dlnvc_dlnR + 5*x**2 - 13/3.*x + 2/3.
    if b**2-4*c >= 0:
        return [(-b +j * (b**2-4*c)**0.5)/2. for j in (-1,1)]
    else:
        if pr: print('no transsonic solutions')
        return None, None

In [ ]:
@njit
def compute_dvdr_dTdr(v, T, r, Mdot, vc2, Lambda):
    cs2 = (gamma * k_Bcgs * T) / (mu * mp_g)
    tflow = r / abs(v)
    mach2 = v**2 / cs2
    rho = Mdot / (4 * np.pi * r**2 * v)
    n = rho / (mu * mp_g)
    tcool = (3 * k_Bcgs * T) / (2 * Lambda * n)

    dlnvdlnr = (2 - (vc2 / cs2) - (tflow / (gamma * tcool))) / (mach2 - 1.0)
    dlnTdlnr = (tflow / tcool) - (2 / 3) * (2 + dlnvdlnr)

    dvdr = (v / r) * dlnvdlnr
    dTdr = (T / r) * dlnTdlnr

    return dvdr, dTdr

In [ ]:
# THE ODE SOLVER ITSELF
def TheODE(r, C, Mdot, Lambdatype, recorder=None):
    v, T = C
    vc2 = vc_interp(r)**2

    # n is computed inside, so use dummy rho/n for Lambda
    rho = Mdot / (4 * np.pi * r**2 * v)
    n = rho / (mu * mp_g)
    Lambda = lf.Lambdacalc(np.log10(T), r, Lambdatype, n)

    # if T > 10**8.16:
    #     print(T)
    dvdr, dTdr = compute_dvdr_dTdr(v, T, r, Mdot, vc2, Lambda)

    if recorder is not None:
        recorder["ra2"].append(r)
        recorder["varray"].append(v)
        recorder["Tarray"].append(T)
        recorder["rhoarray"].append(rho)
        cs2 = (gamma * k_Bcgs * T) / (mu * mp_g)  
        Bern = (v**2 / 2) + (cs2 * 3 / 2) + phi(r)
        recorder["Bern"].append(Bern)
        mach = np.sqrt(v**2 / cs2)
        recorder["Mach"].append(mach)
    return [dvdr, dTdr]

In [ ]:
# CLASSES (THE BANE OF MY EXISTENCE)
class IntegrationResult:
    def __init__(self, res, stop_reason, xval=None, R0=None, v0=None, T0=None, Mdot=None):
        self.res = res
        self._stop_reason = stop_reason
        self.xval = xval
        self.R0 = R0
        self.v0 = v0
        self.T0 = T0
    
    def stopReason(self):
        return self._stop_reason
    
    def Rs(self):
        return self.res.t
    
    def __getitem__(self, key):
        return self.res[key]


# EVENTS LIST
def event_unbound(r, C, Mdot, Lambdatype, _):
    v, T = C
    cs2 = (gamma * k_Bcgs * T) / (mu * mp_g)
    phi_r = phi(r)
    bern = 0.5 * v**2 + 1.5 * cs2 + phi_r
    return bern
event_unbound.terminal = True
event_unbound.direction = 1

def event_lowT(r, C, Mdot, Lambdatype, _):
    T = C[1]
    return T - (10**4.2)
event_lowT.terminal = True
event_lowT.direction = -1 

def event_sonic_point(r, C, Mdot, Lambdatype, _):
    v, T = C
    cs2 = (gamma * k_Bcgs * T) / (mu * mp_g)
    mach = v / np.sqrt(cs2)
    return mach - 1.0
event_sonic_point.terminal = True
event_sonic_point.direction = -1

def event_max_R(r, C, Mdot, Lambdatype, _):
    v, T = C
    return r - (16000*ktc)
event_max_R.terminal = True
event_max_R.direction = 1

def event_overstepdlnv(r, C, Mdot, Lambdatype, _):
    v, T = C

    vc2 = vc_interp(r)**2
    cs2 = (gamma * k_Bcgs * T) / (mu * mp_g)
    
    tflow = r / np.abs(v)
    mach = v / np.sqrt(cs2)

    rho = Mdot / (4 * np.pi * r**2 * v)
    n = rho / (mu * mp_g)
    Lambda = lf.Lambdacalc(np.log10(T), r, Lambdatype, n)
    
    tcool = (3 * k_Bcgs * T) / (2 * Lambda * n)

    dlnvdlnr = (2 - (vc2 / cs2) - (tflow/ (gamma*tcool))) / (mach**2 - 1.0)
    return np.abs(dlnvdlnr) - 50
event_overstepdlnv.terminal = True
event_overstepdlnv.direction = 1 

my_event_list = [
    event_sonic_point,
    event_unbound,
    event_lowT,
    event_max_R,
    event_overstepdlnv
]


event_names = ['sonic point', 'unbound', 'lowT', 'max R reached', 'overstepdlnv' ]

# SHOOTING METHOD
def sonic_point_shooting(Rsonic, Lambdatype, Rmax=16000*ktc, tol=1e-8, epsilon=1e-5, dlnMdlnRInit=-1, x_high=0.99, x_low=0.01, return_all_results=False):
    results = {}
    dlnMdlnRold = dlnMdlnRInit
    
    # x = v_c / 2*c_s is the iterative variable
    while x_high - x_low > tol:
        #INITIAL GUESSES
        x = 0.5 * (x_high + x_low)
        #print(x)
        cs2_sonic = vc_interp(Rsonic)**2 / (2 * x)
        v_sonic = cs2_sonic**0.5
        T_sonic = mu * mp_g * cs2_sonic / (gamma * k_Bcgs)
        tflow_to_tcool = (10/3) * (1 - x)
        
        rho_sonic = lf.rhocalc(v_sonic, tflow_to_tcool, T_sonic, Rsonic, Lambdatype)
        if rho_sonic == False:
            x_high = x
            continue
        Mdot = 4 * np.pi * Rsonic**2 * rho_sonic * v_sonic
        
        dlnTdlnR1, dlnTdlnR2 = dlnTdlnrcalc(Rsonic, x, T_sonic, Lambdatype, pr=True)
        if dlnTdlnR1 is None:
            x_high = x
            continue
        
        dlnMdlnR1, dlnMdlnR2 = [3 - 5*x - 2*dlnTdlnR for dlnTdlnR in (dlnTdlnR1, dlnTdlnR2)]
        if abs(dlnMdlnR1 - dlnMdlnRold) < abs(dlnMdlnR2 - dlnMdlnRold):
            dlnTdlnR = dlnTdlnR1
        else:
            dlnTdlnR = dlnTdlnR2
        
        dlnMdlnR = 3 - 5*x - 2*dlnTdlnR
        
        dlnvdlnR = -1.5 * dlnTdlnR + 3 - 5 * x
        
        T0 = T_sonic * (1 + epsilon * dlnTdlnR)
        v0 = v_sonic * (1 + epsilon * dlnvdlnR)
        R0 = Rsonic * (1 + epsilon)

        # Early checks
        cs2_0 = (gamma * k_Bcgs * T0) / (mu * mp_g)
        mach0 = v0 / np.sqrt(cs2_0)
        if mach0 > 1.0:
            print("starts supersonic")
            x_high = x
            continue

        phi0 = phi(R0)
        bern = 0.5 * v0**2 + 1.5 * cs2_0 + phi0
        
        if bern > 0:
            print(f"starts unbound")
            x_low = x
            continue
        res_raw = solve_ivp(TheODE, [R0, Rmax], [v0, T0], args=(Mdot, Lambdatype, None), method='RK45', 
            atol=1e-5, rtol=1e-5, events=my_event_list, dense_output=True)
        
        if res_raw.status < 0:
            stop_reason = 'integration failure'
        elif any(len(evt) > 0 for evt in res_raw.t_events):
            for idx, t_evt in enumerate(res_raw.t_events):
                if len(t_evt) > 0:
                    stop_reason = event_names[idx]
                    break
        else:
            if res_raw.t[-1] >= Rmax:
                stop_reason = 'max R reached'
            else:
                stop_reason = 'unknown'
        
        res = IntegrationResult(res_raw, stop_reason, xval=x, R0=R0, v0=v0, T0=T0)
        
        #print(f"maximum r = {res.Rs()[-1] / 3.0857e21:.2f} kpc; stop reason: {res.stopReason()}")
        
        if res.stopReason() in ('sonic point', 'lowT', 'overstepdlnv'):
            x_high = x
            continue
        elif res.stopReason() == 'unbound':
            x_low = x
            continue
        elif res.stopReason() == 'max R reached':
            dlnMdlnRold = dlnMdlnR
            results[x] = res
            print(f"x = {x}, Rsonic = {Rsonic/ktc} Mdot = {Mdot/Mdot1}")
            break
        else:
            print(f"Warning: Unexpected stopReason '{res.stopReason()}' — stopping loop.")
            break
    
    if return_all_results:
        return results
    if len(results) == 0:
        print("no result reached maximum R")
        return None
    else:
        return results[x]

In [ ]:
def find_converged_x(Rsonic, Lambdatype):
    result = sonic_point_shooting(Rsonic, Lambdatype)
    if result is None:
        print("No solution reached max radius.")
        return None
    return result 


def find_mdot(Rsonic, Lambdatype, result=None):
    if result is None:
        result = sonic_point_shooting(Rsonic, Lambdatype)
        if result is None:
            return np.nan
    x = result.xval
    cs2_sonic = vc_interp(Rsonic)**2 / (2 * x)
    v_sonic = cs2_sonic**0.5
    T_sonic = mu * mp_g * cs2_sonic / (gamma * k_Bcgs)
    tflow_to_tcool = (10 / 3) * (1 - x)
    rho_sonic = lf.rhocalc(v_sonic, tflow_to_tcool, T_sonic, Rsonic, Lambdatype)
    if rho_sonic is False:
        return np.nan
    Mdot = 4 * np.pi * Rsonic**2 * rho_sonic * v_sonic
    return Mdot

def postprocess(b, Mdot, Lambdatype):
    b_result = sonic_point_shooting(b, Lambdatype)
    x = b_result.xval
    R0 = b_result.R0
    v0 = b_result.v0
    T0 = b_result.T0
    Rmax = 16000 * ktc

    recorder = {"ra2": [], "varray": [], "Tarray": [], "rhoarray": [], "Bern": [], "Mach": []}

    res = solve_ivp(TheODE, [R0, Rmax], [v0, T0], args=(Mdot, Lambdatype, recorder), method='RK45', max_step=Rmax / 100,
        atol=1e-5, rtol=1e-5, dense_output=True)

    return x, R0, v0, T0, recorder

In [ ]:
def BrentLooper(Mdot, Rsonlow, Rsonhigh, Lambdatype, tol=(2e-6 * Mdot1)):
    target = Mdot
    a, b = Rsonlow, Rsonhigh
    fa = find_mdot(a, Lambdatype) - target
    fb = find_mdot(b, Lambdatype) - target

    if fa * fb >= 0:
        raise ValueError("Not bounded correctly!")

    if abs(fa) < abs(fb):
        a, b = b, a
        fa, fb = fb, fa

    c = a
    fc = fa
    d = e = b - a
    mflag = True

    while abs(b - a) > tol:
        if abs(fb) < tol:
            return postprocess(b, Mdot, Lambdatype)
        if fa != fc and fb != fc:
            # Inverse quadratic interpolation
            s = (a * fb * fc / ((fa - fb) * (fa - fc)) +
                 b * fa * fc / ((fb - fa) * (fb - fc)) +
                 c * fa * fb / ((fc - fa) * (fc - fb)))
        else:
            # Secant method
            s = b - fb * (b - a) / (fb - fa)

        if a < b:
            cond1 = not ((3 * a + b) / 4 < s < b)
        else:
            cond1 = not (b < s < (3 * a + b) / 4)

        cond2 = mflag and abs(s - b) >= abs(b - c) / 2
        cond3 = (not mflag) and abs(s - b) >= abs(c - d) / 2
        cond4 = mflag and abs(b - c) < tol
        cond5 = (not mflag) and abs(c - d) < tol

        if cond1 or cond2 or cond3 or cond4 or cond5:
            s = (a + b) / 2
            mflag = True
        else:
            mflag = False

        fs = find_mdot(s, Lambdatype) - target
        d, c = c, b
        fd, fc = fc, fb

        if fa * fs < 0:
            b = s
            fb = fs
        else:
            a = s
            fa = fs

        if abs(fa) < abs(fb):
            a, b = b, a
            fa, fb = fb, fa
    return postprocess(b, Mdot, Lambdatype)

In [ ]:
lambdatypes = [0.1]
def profile_mdot_looper():
    for Lambdatype in lambdatypes:
        try:
            x, R0, v0, T0, recorder = BrentLooper(Mdot1 * 2700, 1 * ktc, 10 * ktc, Lambdatype)
        except Exception as e:
            print(f"Failed for Λ={Lambdatype}: {e}")

%prun -s cumulative profile_mdot_looper()

In [ ]:
qgrid = np.linspace(1*ktc, 10*ktc, 30)
newgrid = [find_mdot(r, 0.6) for r in qgrid]

plt.scatter(qgrid, newgrid)
plt.xlabel("r")
plt.ylabel("Ṁ")
plt.title("Ṁ vs r")
plt.show()

In [ ]:
x_values = np.array([1069.01920, 586.29466, 355.40590, 230.31321, 159.55064, 126.31336, 108.70059, 88.97651, 76.56988, 62.67603, 47.19685, 40.61586, 
    21.90684, 18.23348, 14.67799, 11.62030, 8.18547])
y_values = np.array([2413.28546, 1405.48459, 656.73673, 360.18178, 201.53377, 146.29093, 119.74596, 90.47357, 72.58879, 54.84417, 38.24800, 31.30777, 
    15.84893, 13.23546, 8.86801, 3.60182, 2.27259])

Tmpx = np.array([1080.61262, 590.26057, 353.85191, 229.22889, 157.99859, 125.21118, 107.22672, 89.02151, 77.42637, 63.29171, 53.36699, 47.14120, 41.00102, 
    34.57168, 27.82559, 22.39589, 17.74836, 14.73500, 11.49757, 8.30218, 3.32572])

Tmpy = np.array([10.29864, 13.44796, 12.94118, 12.90498, 11.60181, 10.91403, 10.84163, 9.61086, 9.17647, 8.34389, 8.27149, 7.83710, 7.51131, 6.93213,
    6.31674, 6.17195, 6.06335, 5.33937, 3.34842, 2.29864, 2.15385])*kevtk

In [ ]:
lambdatypes = [0.1, 0.3, 1, -1, -2]
colors     = ['#cc2200', "#cc6622", '#ccaa33', '#aacc33', "#227799"]
labels = [rf"$\dot{{Lambda}}={Lambda}$" for Lambda in lambdatypes]

results = []
for Lambdatype in lambdatypes:
    try:
        x, R0, v0, T0, recorder = BrentLooper(Mdot1 * 2700, 1 * ktc, 12 * ktc, Lambdatype)
        results.append((Lambdatype, recorder))
    except Exception as e:
        print(f"Failed for Λ={Lambdatype}: {e}")

# --- Create subplots ---
fig, axes = plt.subplots(1, 5, figsize=(18, 5), sharex=True)

for (Lambdatype, recorder), color, label in zip(results, colors, labels):
    r_kpc = np.array(recorder["ra2"]) / ktc
    T     = np.array(recorder["Tarray"])
    v     = np.array(recorder["varray"])
    rho   = np.array(recorder["rhoarray"])
    K     = (rho/mp_g)**(-2/3) * k_Bcgs * T * etkv
    Bern = np.array(recorder["Bern"])
    Mach = np.array(recorder["Mach"])

    axes[0].plot(r_kpc, T, color=color, label=label)
    axes[1].plot(r_kpc, v, color=color, label=label)
    axes[2].plot(r_kpc, K, color=color, label=label)
    axes[3].plot(r_kpc, Bern, color=color, label=label)
    axes[4].plot(r_kpc, Mach, color=color, label=label)

# --- Format each panel ---
for ax, ylabel, title in zip(
    axes,
    ['Temperature (K)', 'Velocity (cm/s)', 'Entropy (keV cm²)', 'Bernoulli Parameter', 'Mach number'],
    ['T vs Radius', 'v vs Radius', 'K vs Radius', 'B vs Radius', 'Mach number vs. Radius']
):
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel('Radius (kpc)')
    ax.set_xlim(1, 2500)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.grid(True, which='both', ls='--')
    ax.legend()

axes[0].set_ylim(1e7, 3e8)
axes[0].scatter(Tmpx, Tmpy, s=20, marker='x', color='black', label='Phoenix data')
axes[2].scatter(x_values, y_values, s=20, marker='x', color='black', label='Phoenix data')
axes[2].legend()
axes[2].set_ylim(2, 3e3)
axes[3].set_yscale('linear')
axes[3].set_ylim(None, 0)
axes[4].set_ylim(None, 1)

plt.tight_layout()
plt.show()

In [ ]:
Mdotrange = [2000, 2700, 3400, 4100, 4800]
colors     = ['purple', 'crimson', 'tomato', 'darkorange', 'goldenrod']
labels = [rf"$\dot{{M}}={m}$" for m in Mdotrange]

results = []
for Mdot in Mdotrange:
    try:
        x, R0, v0, T0, recorder = BrentLooper(Mdot1 * Mdot, 1 * ktc, 15 * ktc, 0.3)
        results.append((Mdot, recorder))
    except Exception as e:
        print(f"Failed for Mdot={Mdot}: {e}")

# --- Create subplots ---
fig, axes = plt.subplots(1, 4, figsize=(18, 5), sharex=True)

for (Mdot, recorder), color, label in zip(results, colors, labels):
    r_kpc = np.array(recorder["ra2"]) / ktc
    T     = np.array(recorder["Tarray"])
    rho   = np.array(recorder["rhoarray"])
    K     = (rho/mp_g)**(-2/3) * k_Bcgs * T * etkv
    Bern = np.array(recorder["Bern"])
    Mach = np.array(recorder["Mach"])

    axes[0].plot(r_kpc, T, color=color, label=label)
    axes[1].plot(r_kpc, K, color=color, label=label)
    axes[2].plot(r_kpc, Bern, color=color, label=label)
    axes[3].plot(r_kpc, Mach, color=color, label=label)

# --- Format each panel ---
for ax, ylabel, title in zip(axes, ['Temperature (K)', 'Entropy (keV cm²)', 'Bernoulli Parameter', 'Mach number'],
    ['T vs Radius', 'K vs Radius', 'B vs Radius', 'Mach number vs. Radius']):
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel('Radius (kpc)')
    ax.set_xlim(1, 2500)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.grid(True, which='both', ls='--')
    ax.legend()

axes[0].set_ylim(1e7, 3e8)
axes[0].scatter(Tmpx, Tmpy, s=20, marker='x', color='black', label='Phoenix data')
axes[1].scatter(x_values, y_values, s=20, marker='x', color='black', label='Phoenix data')
axes[1].set_ylim(2, 3e3)
axes[2].set_yscale('linear')
axes[2].set_ylim(None, 0)
axes[3].set_ylim(None, 1)

plt.tight_layout()
plt.show()

In [ ]:
y = np.log10(np.array([3324.598, 1522.680, 1553.475, 1297.309, 725.888, 594.173, 374.898, 346.043, 214.010, 193.623, 155.348, 137.762, 127.159, 112.765,
    96.075, 85.199, 78.642, 69.739, 57.085, 51.647, 39.811, 35.304, 33.246, 30.079, 16.830, 14.925, 14.055, 12.716, 9.607, 8.185,
    4.228, 3.008, 2.563, 1.975]))

y_pairs = y.reshape(-1, 2)

# Compute symmetric error for each pair
errors = np.abs(y_pairs[:, 0] - y_pairs[:, 1]) / 2
stdvi = errors**2

y_kvalues = np.log10(np.array([2413.28546, 1405.48459, 656.73673, 360.18178, 201.53377, 146.29093, 119.74596, 90.47357, 72.58879, 54.84417, 38.24800, 31.30777, 
    15.84893, 13.23546, 8.86801, 3.60182, 2.27259]))

In [ ]:
def ChiSquaredCalc(Mdot, Lambdatype):
    x, R0, v0, T0, recorder = BrentLooper(Mdot*Mdot1, 1*ktc, 30*ktc, Lambdatype)
    rA = np.array(recorder["ra2"])
    TA = np.array(recorder["Tarray"])
    rhoA = np.array(recorder["rhoarray"])
    KA = (rhoA / (mu*mp_g))**(-2/3) * k_Bcgs * TA * etkv
    Kvals = []
    for j in range(len(x_values)):
        idx = np.argmin(np.abs(rA - x_values[j]*ktc))
        Kvals.append(np.log10(KA[idx]))
    sqdiff = (Kvals - y_values)**2
    return sqdiff / stdvi

In [ ]:
all_n = []
all_chi2 = []
for i in range(1, 21):
    n = i*40 + 3975
    x, R0, v0, T0, recorderA = BrentLooper(Mdot1*n, 1*ktc, 8*ktc, 0.3)
    rA = np.array(recorderA["ra2"])
    TA = np.array(recorderA["Tarray"])
    rhoA = np.array(recorderA["rhoarray"])
    KA = (rhoA/(mu*mp_g))**(-2/3) * k_Bcgs * TA * etkv
    Kvals = []
    for j in range(len(x_values)):
        idx = np.argmin(np.abs(rA - x_values[j]*ktc))
        Kvals.append(KA[idx])
    Kvals = np.log10(np.array(Kvals))
    sqddiff = (Kvals - y_values)**2
    Chi2 = (Kvals - y_kvalues)**2 / stdvi
    all_n.append(n)
    all_chi2.append(np.sum(Chi2)/16)

    print(np.sum(Chi2)/16, n)

plt.plot(all_n, all_chi2, marker='o')
plt.xlabel("Mdot scaling factor (n)")
plt.ylabel("Reduced χ²")
plt.title("Model Fit vs. Mass Flux")
plt.grid(True)
plt.show()

In [ ]:
lambdatypes = [0.3, -2]
def profile_mdot_looper():
    for Lambdatype in lambdatypes:
        try:
            x, R0, v0, T0, recorder = TheMdotLooper(Mdot1 * 2700, 1 * ktc, 6 * ktc, Lambdatype)
        except Exception as e:
            print(f"Failed for Λ={Lambdatype}: {e}")

%prun -s cumulative profile_mdot_looper()

In [ ]:
Mdotpredx = [3.34829, 8.35576, 11.58323, 15.28964, 18.29833, 22.62611, 27.97747, 34.59450, 41.40200, 47.95690, 54.64981, 64.34438, 
    78.27391, 92.15927, 108.50781, 127.75649, 160.57346, 233.77299, 351.64083, 602.75642, 1085.07809]

Mdotpredy = [788.04628, 1697.79425, 2154.43469, 1838.09442, 2043.35972, 2525.21785, 3290.34456, 3562.24789, 3657.78682, 3562.24789, 
    3469.20437, 3290.34456, 3120.70613, 2882.50530, 2733.89361, 2525.21785, 2271.54759, 1653.44900, 1141.48777, 654.77480, 365.77868]

Mdotpredx = np.array(Mdotpredx)*ktc
Mdotpredy = np.array(Mdotpredy)
Mdot_obsarray = np.array(recorder["Mdotobsarray"])
Mdot_obsarray = (Mdot_obsarray * 60 * 60 * 24 * 365)/(1.988e33)

Mcool = np.array(recorder["Mcool"])
Mcool = (Mcool * 60 * 60 * 24 * 365) / (1.988e33)

plt.scatter(ra2, Mdot_obsarray, s=0.1, label='Observational Mdot')
plt.scatter(ra2, Mcool, s=0.1, label='Voit Mdot Approx.')
plt.scatter(Mdotpredx, Mdotpredy, s=4, label = 'Phoenix Mdot (Prediction)')
plt.xlabel('Radius (cm)')
plt.ylabel('Observational Mdot')
plt.xscale('log')
plt.yscale('log')
plt.title('Observational Mdot as a function of radius.')
plt.legend()
plt.xlim(0.5*ktc, 16000*ktc)
plt.ylim(0.1, 9000)
plt.grid(True)
#plt.savefig('dlnvdlnr components.png', transparent=True)
plt.show()